# Multimodal Accessibility Analysis *At Scale*: 
## Measuring Access to Covid Testing Sites in California

This notebook calculates access to Covid-19 testing sites for every block group in California. Apart from the locations of test sites, all data are collected on the fly. To account for the massive size of the study area (all of CA) the analysis is chunked at the county level; access is computed for the blockgroups in each county (with both supply and demand data buffered 8km beyond the county borders to mitigate edge effects), then the counties are recombined into a statewide set. 

In [1]:
import cenpy
import osmnet
import os
import importlib
import pandas as pd
import numpy as np
import pandana as pdna
import urbanaccess as ua
import geopandas as gpd
import matplotlib.pyplot as plt
from geosnap import Community, datasets
from access import access as Access

/Users/preetijuturu/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/Users/preetijuturu/opt/anaconda3/lib/python3.7/site-packages/geosnap/_data.py:123: UserWarning: Unable to locate local census data. Streaming instead.
If you plan to use census data repeatedly you can store it locally with the io.store_census function for better performance
  "Unable to locate local census data. Streaming instead.\n"
Loading manifest: 100%|██████████| 5/5 [00:00<00:00, 1117.83entries/s]


In [2]:
import contextily as ctx

In [3]:
from access import weights as acweights
from healthacc.travel_matrix import compute_travel_cost_adjlist

ModuleNotFoundError: No module named 'healthacc'

In [4]:
from segregation.util import project_gdf

ModuleNotFoundError: No module named 'segregation'

## Testing Locations (testing supply)

In [5]:
test_locations = gpd.read_file("../data/accessbility/Testing_Locations_7_15.shp")

DriverError: ../data/accessbility/Testing_Locations_7_15.shp: No such file or directory

In [6]:
test_locations['count'] = 1

NameError: name 'test_locations' is not defined

## Census Data (testing demand)

In [7]:
counties = datasets.counties()

In [8]:
names = pd.read_excel('../data/all-geocodes-v2017.xlsx', skiprows=4, dtype=object)

FileNotFoundError: [Errno 2] No such file or directory: '../data/all-geocodes-v2017.xlsx'

In [9]:
names['geoid'] = names['State Code (FIPS)'] + names['County Code (FIPS)']

NameError: name 'names' is not defined

In [10]:
counties = counties.merge(names, on='geoid')

NameError: name 'names' is not defined

In [11]:
ca_counties = counties[counties.geoid.str.startswith('06')]

In [12]:
ca_counties.head()

,geoid,geometry
8,06091,"POLYGON ((-120.65559 39.69356, -120.65552 39.6..."
325,06067,"POLYGON ((-121.18857 38.71431, -121.18731 38.7..."
329,06083,"MULTIPOLYGON (((-120.58190 34.09856, -120.5822..."
346,06009,"POLYGON ((-120.63093 38.34110, -120.63057 38.3..."
394,06111,"MULTIPOLYGON (((-119.63630 33.27304, -119.6360..."


In [13]:
ca_blkgrps= gpd.read_file("../data/accessbility/CA_BG_Income.shp")

DriverError: ../data/accessbility/CA_BG_Income.shp: No such file or directory

In [14]:
ca_blkgrps.crs

NameError: name 'ca_blkgrps' is not defined

In [15]:
ca_blkgrps = ca_blkgrps.to_crs(4326)

NameError: name 'ca_blkgrps' is not defined

In [16]:
counties = [county[-3:] for county in ca_counties.geoid.tolist()]

Our blockgroup geometries seem to contain income data, but we also need population data to pass to `access`. We'll grab blockgroup-level population data using `cenpy`, but since blockgroups [arent implemented]() in the products api yet, we need to loop over each county

In [ ]:
if not os.path.exists('../data/ca_blockgroup_population.csv'):

    cadict = []
    conn = cenpy.products.APIConnection("ACSDT5Y2017")   
    for county in counties:
        data = conn.query(["B01001_001E"], geo_unit = 'block group', geo_filter = {"state": "06", "county": f'{county}'})
        cadict.append(data)
    gdf = pd.concat(cadict)
    gdf.to_csv('../data/ca_blockgroup_population.csv')
else:
    gdf = pd.read_csv('../data/ca_blockgroup_population.csv', converters={'state':str, 'county':str, 'tract':str, 'block group': str})

In [ ]:
gdf['geoid'] = gdf.state + gdf.county + gdf.tract + gdf['block group']

In [ ]:
blockgroups = ca_blkgrps.merge(gdf, left_on='GEOID10', right_on='geoid')

In [ ]:
blockgroups.plot()

## OSM Data (pedestrian network)

To download the networks for each county we need to do a little bit of laborious data processing. The study area of *all* of california is too large of a problem size to deal with in a single go (we're likely talking  about a point-to-point travel matrix of billions of OSM intersections...) so we split the data and process by county. To mitigate edge effects though, we need to buffer each county beyond the travel threshold we care about and operate on the buffered data before clipping it back to the proper spatial extent. OSM handles data in lat/long, so we first need to iterate over each california county and project it into the proper CRS. California is a big state, so it falls into two different UTM zones (10 and 11). I don't want to lookup the proper CRS for each county, so here we use the `utm` library to:
- loop over each county
- project it to utm
- buffer it out 8000m
- project it back to WGS and collect the bounding box
- send the bbox to OSM to download the network and store it

In [ ]:
def determine_utm(polygon):
    
    lon = polygon.centroid.x
    lat = polygon.centroid.y
    
    zone = utm.from_latlon(lat,lon)
    crs = crs = f"+proj=utm +zone={str(zone[2])} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"
    
    return crs

In [ ]:
def buffer_bbox(polygon):
    
    crs = determine_utm(polygon)
    gdf = gpd.GeoDataFrame(pd.Series([polygon], name='geometry'))
    gdf.crs=4326
    gdf = gdf.to_crs(crs).buffer(8000)
    gdf = gdf.to_crs(4326)
    
    return gdf.total_bounds
    

In [ ]:
for county in ca_counties.iterrows():
    fname = f"../data/networks/osmnet_{county[1].geoid}.h5"
    if not os.path.exists(fname):
        bbox = buffer_bbox(county[1].geometry)
        nodes, edges = osmnet.network_from_bbox(bbox=tuple(bbox))
        net = pdna.Network(nodes["x"],
                           nodes["y"],
                           edges["from"],
                           edges["to"],
                           edges[["distance"]])
        net.save_hdf5(fname)
    else:
        pass
    

## GTFS Data (transit network)

In my first attempt, I used the builtin downloader but the results were unsatisfactory, so I wrote a couple functions to hit the transitland api instead, which works better

~~Brute force: add all feeds for california*, then clip the network to an extent using the buffered bboox for each county. GTFS data are pretty small so this is easy and fast~~

~~two things to note here. First, urbanaccess is setup to query from GTFS Data Exchange which was shut down a few years ago, so the data may be stale. Second, we're doing a string search for California, which may not necessarily be in every CA transit feed name. TransitLand has popped up to replace  GTGS Data Exchange, and its api allows searching by geograpic extent via bounding boxes. There's an experimental (but stale) branch of ua that implements a transitland search here https://github.com/UDST/urbanaccess/blob/kuanb-gtfs-feeds-query-util/urbanaccess/gtfsfeeds.py but i havent tested it. ~~

In [ ]:
from healthacc import feeds_from_bbox

In [ ]:
# these throw a 403. I think the call to `requests` inside urbanaccess needs to be beefed up with user agent and headers and such 
# <https://stackoverflow.com/questions/38489386/python-requests-403-forbidden>
fourofours = ['metro', 'laketahoe','southwestpoint','metrolinktrains', 'riversidetransitagency','blue','tidelinewatertaxi','countyconnection']

# throw  401 for not having an API key for 511.org
fouroones = ['capitolcorridor','angelislandtiburonferry', 'acealtamontcorridorexpress', 'goldengateferry', 'alcatrazhornblowerferry', 'smart', \
            'unioncitytransit', 'vacavillecitycoach', 'vinenapacounty', 'sanfranciscobayferry', 'americancanyontransit', 'dumbartonexpress']
#this one times out
tout = ['trideltatransit']
# these are corrupt zips or bad GTFS data
# usually missing calendar_dates, but other errors too
incorrect = ['commuteorgshuttle', 'culvercitybus', 'airportexpressinc', 'madera','anaheim','vctc','goldengatetransit','kerncounty','delnorte',\
            'yosemite','longbeachtransit', 'getbus', 'avalon', 'amtrak', 'petalumatransit'] 

feederrors = fourofours + fouroones + tout + incorrect




In [ ]:
def build_network(county, bbox):
    COUNTY_PATH = f"../data/counties/{county}"
    if os.path.exists(f"../data/matrices/{county}_adj.parquet"):
        print(f"network adj list for {county} already exists")
        combined_net=''
    else:
        osm_network = pdna.Network.from_hdf5(f"../data/networks/osmnet_{county}.h5")

        try:
            # download gtfs data if we need it
            if os.path.exists(f"{COUNTY_PATH}/{county}_gtfs.h5"):
                loaded_feeds = ua.gtfs.network.load_processed_gtfs_data(f"{county}_gtfs.h5", dir=COUNTY_PATH,)
            else:   
                # get GTFS feeds for the area
                feeds = feeds_from_bbox(bbox)
                for feed in list(feeds.keys()):
                    if feed in feederrors:
                        feeds.pop(feed)
                if len(ua.gtfsfeeds.feeds.to_dict()['gtfs_feeds'])>0:
                    ua.gtfsfeeds.feeds.remove_feed(remove_all=True)  #feeds object is global so reset it from last iter
                ua.gtfsfeeds.feeds.add_feed(feeds)
                ua.gtfsfeeds.download()
                loaded_feeds = ua.gtfs.load.gtfsfeed_to_df(f"{COUNTY_PATH}/gtfsfeed_text/",
                                                           bbox=bbox,
                                                           remove_stops_outsidebbox=True)
                ua_to_h5(loaded_feeds, f"{COUNTY_PATH}/{county}_gtfs.h5")

            # Create transit and OSM networks and combine them into a single multimodal net
            ua.create_transit_net(gtfsfeeds_dfs=loaded_feeds,
                                           day='monday',
                                           timerange=['07:00:00', '10:00:00'],
                                           calendar_dates_lookup=None)
            osm_network.nodes_df['id'] = osm_network.nodes_df.index
            ua_osm = ua.create_osm_net(osm_edges=osm_network.edges_df,
                                      osm_nodes=osm_network.nodes_df,
                                      travel_speed_mph=3)
            urbanaccess_net = ua.ua_network
            ua.integrate_network(urbanaccess_network=urbanaccess_net,
                                 headways=False)
            combined_net = pdna.Network(urbanaccess_net.net_nodes["x"],
                                        urbanaccess_net.net_nodes["y"],
                                        urbanaccess_net.net_edges["from_int"],
                                        urbanaccess_net.net_edges["to_int"],
                                        urbanaccess_net.net_edges[["weight"]])

        except KeyError:  # fallback to ped-only network if no gtfs in that county
            combined_net = pdna.Network(osm_network.nodes_df["x"],
                                        osm_network.nodes_df["y"],
                                        osm_network.edges_df["from"],
                                        osm_network.edges_df["to"],
                                        osm_network.edges_df[["weight"]])
    return combined_net

In [ ]:
def ua_to_h5(loaded_feeds, path):
    
    hdf = pd.HDFStore(path)
    hdf['calendar'] =loaded_feeds.calendar
    hdf['calendar_dates'] =loaded_feeds.calendar_dates
    hdf['headways'] =loaded_feeds.headways
    hdf['routes'] =loaded_feeds.routes
    hdf['stop_times'] =loaded_feeds.stop_times
    hdf['stop_times_int'] =loaded_feeds.stops
    hdf['stops'] =loaded_feeds.stops
    hdf['trips'] = loaded_feeds.trips
    hdf.close()

## Calculating Accessibility

In [ ]:
def prepare_data(county):
    
    df = ca_blkgrps[ca_blkgrps.GEOID10.str[:5]==county].copy()
    boundary = project_gdf(df).buffer(8000)
    utm = boundary.crs
    boundary = boundary.to_crs(4326).unary_union
    
    # this part is expensive
    buf_testint = test_locations[test_locations.intersects(boundary)]    
    buf_blkgrps = ca_blkgrps[ca_blkgrps.centroid.intersects(boundary)]

    geoms = buf_blkgrps[['GEOID10', 'geometry']]
    
    
    dests = gpd.sjoin(geoms, buf_testint, how='left', op='intersects').groupby('GEOID10').sum().merge(geoms, on='GEOID10')
    
    buf_blkgrps = buf_blkgrps.merge(dests[['GEOID10', 'count']], on='GEOID10')
    buf_blkgrps = buf_blkgrps.merge(blockgroups[['GEOID10', 'B01001_001E']], on='GEOID10', how='left')
    buf_blkgrps =  gpd.GeoDataFrame(buf_blkgrps)
    buf_blkgrps.crs = 4326

    
    return buf_blkgrps



In [ ]:
def compute_access(county, blkgrps, network):
    if os.path.exists(f"../data/matrices/{county}_adj.parquet"):
        adjlist = pd.read_parquet(f"../data/matrices/{county}_adj.parquet")
    else:
        adjlist = compute_travel_cost_adjlist(blkgrps, blkgrps, network, reindex_dest='GEOID10', reindex_orig='GEOID10')
        adjlist.assign(temp=adjlist.origin+adjlist.destination).drop_duplicates(subset='temp', inplace=True)
        adjlist.to_parquet(f"../data/matrices/{county}_adj.parquet")
    ac_test=Access(demand_df = blkgrps,
            demand_index = 'GEOID10',
            demand_value = 'B01001_001E',
            supply_df    = blkgrps,
            supply_index = 'GEOID10',
            cost_df=adjlist.replace(0.0,1.0),  # gravity chokes if travel time is 0
            cost_origin='origin',
            cost_dest='destination',
            cost_name='cost',
            supply_value='count',
            neighbor_cost_df     = adjlist.replace(0.0,1.0),
            neighbor_cost_origin = 'origin',
            neighbor_cost_dest   = 'destination',
            neighbor_cost_name   = 'cost'
      )
    try: 
        ac_test.raam(name = "raam", tau = 60);
        ac_test.access_df["raam_count"] = 1 / ac_test.access_df["raam_count"] 
    except:
        print('raam failed')
    ac_test.two_stage_fca(name ="2sfca", max_cost = 60,)
    ac_test.enhanced_two_stage_fca(name = "g2sfca", weight_fn = gaussian)
    ac_test.three_stage_fca(name = "3sfca")
    ac_test.weighted_catchment(name = "catch_gravity", weight_fn = gravity)
    ac_test.weighted_catchment(name = "catch_gaussian", weight_fn = gaussian)
    ac_test.fca_ratio(name = "fca60",      max_cost = 30)
    ac_test.fca_ratio(name = "fca120",      max_cost = 60) 
    
    return ac_test.access_df[ac_test.access_df.index.str.startswith(county)]


In [ ]:
from urbanaccess.config import settings

In [ ]:
settings.to_dict()

In [ ]:
len(ua.gtfsfeeds.feeds.to_dict()['gtfs_feeds'])

In [ ]:
from access import weights as acweights

In [ ]:
gravity = acweights.gravity(scale = 60, alpha = -1)
gaussian = acweights.gaussian(60)

In [ ]:
settings.log_console=False # turn off urbanccess verbosity

This cell does all the computation and it will take awhile to run. Despite pandana's impressive speed, this is a lot of data to crunch. Most counties are done in a minute or two, but LA takes about an hour on its own.

In [ ]:
ca_statewide = []
for index, row in ca_counties.iterrows():
    try:
        if not os.path.exists(f"../data/counties/{row.geoid}_access.parquet"):

            gdf = prepare_data(row.geoid)
            network = build_network(row.geoid, gdf.total_bounds)
            access = compute_access(row.geoid, gdf.drop_duplicates(subset=['GEOID10']), network)
            access.to_parquet(f"../data/counties/{row.geoid}_access.parquet")
        else:
            access = pd.read_parquet(f"../data/counties/{row.geoid}_access.parquet")
        ca_statewide.append(access)
    except:
        print(f"{row.geoid} failed")
statewide_access = pd.concat(ca_statewide)
statewide_access.to_parquet("../data/ca_statewide_access.parquet")

In [ ]:
statewide_access = pd.read_parquet('../data/ca_statewide_access.parquet')

In [ ]:
blockgroups = blockgroups.merge(statewide_access, left_on='GEOID10', right_index=True)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15,10))
blockgroups.plot('g2sfca_count', cmap='blues', scheme='quantiles', k=8, ax=axs[0])
blockgroups[blockgroups.GEOID10.str.startswith('06073')].plot('g2sfca_count', cmap='blues', scheme='quantiles', k=8, ax=axs[1])
for ax in axs:
    ax.axis('off')
fig.suptitle('Access to COVID Test Sites', fontsize=30)
plt.tight_layout()
plt.savefig('../figures/ca_example.png', dpi=200)

In [ ]:
measures = ['raam_count', '2sfca_count', 'g2sfca_count', '3sfca_count', 'catch_gravity_count', 'catch_gaussian_count', 'fca60_count', 'fca120_count']

In [ ]:
blockgroups = blockgroups.to_crs(3857)

In [ ]:
for idx, county in ca_counties.iterrows():
    fig, ax = plt.subplots(2,4, figsize=(24,16))
    axs=ax.flatten()
    df = blockgroups[blockgroups.GEOID10.str.startswith(f"{county.geoid}")]
    for i, col in enumerate(measures):
        try:
            df.plot(col, cmap='Blues', scheme='quantiles', k=8, ax=axs[i], alpha=0.6)
            ctx.add_basemap(ax=axs[i], source=ctx.providers.Stamen.TonerLite)
            axs[i].axis('off')
            axs[i].set_title(f'{col[:-6]}')
            fig.suptitle(f"{county['Area Name (including legal/statistical area description)']}", fontsize=30)
            plt.tight_layout()
            plt.savefig(f'../figures/counties/{county.geoid}.png', dpi=200)
        except:
            #raise
            print(f"{col} failed")
